# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.mixture import GaussianMixture
from smart_open import open
from stop_words import get_stop_words
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [7]:
newsgroups_df = pd.read_csv("../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [8]:
with open("../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [9]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["gmm"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]
covariance_types = config["clustering"]["gmm"]["covariance_types"]

In [10]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

In [11]:
texts = newsgroups_df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)
#     登場頻度80%以上の単語を削除
dictionary.filter_extremes()

100%|██████████| 120000/120000 [00:00<00:00, 177756.69it/s]


In [12]:
label = newsgroups_df["class"].to_numpy()

In [ ]:
pred_path = "../../Clustering/data/AgNews/doc2vec/GMM/pred/"

for vector_dim in tqdm(vector_dims):
    coherences = []
    for model_num in range(model_nums):
        coherence = []
        for covariance_type in covariance_types:
            pred = np.loadtxt(
                f"{pred_path}{vector_dim}/{covariance_type}/{model_num}.csv", delimiter=","
            )

            

 11%|█         | 1/9 [06:10<49:23, 370.46s/it]